In [1]:
import pandas as pd
from dotenv import load_dotenv
import requests
from datetime import datetime, timedelta

# Load environment variables
load_dotenv(
    dotenv_path="/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env"
)

from api_helpers.config import config
from api_helpers.clients import get_postgres_client

pg = get_postgres_client()

2025-11-15T05:32:56Z | INFO - Logging configuration initialized with level: INFO


# Premier League Historical Data Download

This notebook downloads historical Premier League football data from Betfair's Historical Data service, including:
- Match Odds (In-play and ante-post)
- Over/Under markets
- Correct Score
- Asian Handicap
- And other football markets

**Note:** You need a Betfair Historical Data subscription to download data files.

## Prerequisites

1. **Betfair Account**: You need a Betfair account with API access
2. **Historical Data Subscription**: Subscribe at [Betfair Historical Data](https://historicdata.betfair.com/)
3. **API Credentials**: 
   - App Key (from Betfair Developer Portal)
   - Username & Password
   - SSL Certificates

## Data Types

- **Match Odds**: Home/Draw/Away markets (includes ante-post)
- **Over/Under**: Total goals markets
- **Correct Score**: Exact score predictions
- **BTTS**: Both Teams To Score
- **Asian Handicap**: Handicap betting

## Ante-Post vs In-Play

- **Ante-Post**: Markets that open days/weeks before the event
- **In-Play**: Markets during the match
- Historical data includes the full lifecycle from market opening to settlement

In [ ]:
from dataclasses import dataclass
from datetime import datetime, timedelta
from time import sleep
from typing import Literal

import betfairlightweight
import numpy as np
import pandas as pd
import requests
from api_helpers.helpers.logging_config import D, I
from api_helpers.helpers.time_utils import get_uk_time_now, make_uk_time_aware

# Premier League Event Type ID is 1 (Soccer)
# Competition ID for Premier League is typically found via list_competitions
MARKET_FILTER = betfairlightweight.filters.market_filter(
    event_type_ids=["1"],  # Soccer
    competition_ids=["10932509"],  # Premier League (you may need to verify this)
    market_type_codes=["MATCH_ODDS", "OVER_UNDER_25"],  # Common football markets
    market_start_time={
        "from": (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%dT%TZ"),  # Last year
        "to": datetime.now().strftime("%Y-%m-%dT%TZ"),
    },
)

MARKET_PROJECTION = [
    "COMPETITION",
    "EVENT",
    "EVENT_TYPE",
    "MARKET_START_TIME",
    "MARKET_DESCRIPTION",
    "RUNNER_DESCRIPTION",
    "RUNNER_METADATA",
]

PRICE_PROJECTION = betfairlightweight.filters.price_projection(
    price_data=betfairlightweight.filters.price_data(ex_all_offers=True)
)


@dataclass(frozen=True)
class BetFairCancelOrders:
    market_ids: list[str]


@dataclass(frozen=True)
class BetFairOrder:
    size: float
    price: float
    selection_id: str
    market_id: str
    side: Literal["BACK", "LAY"]
    strategy: str


@dataclass
class BetfairCredentials:
    username: str
    password: str
    app_key: str
    certs_path: str


@dataclass
class BetfairHistoricalDataParams:
    from_day: int
    from_month: int
    from_year: int
    to_day: int
    to_month: int
    to_year: int
    market_types_collection: list[str]
    countries_collection: list[str]
    file_type_collection: list[str]


class BetFairFootballClient:
    """
    Betfair client for football data
    """

    def __init__(
        self,
        username: str,
        password: str,
        app_key: str,
        certs_path: str,
    ):
        self.username = username
        self.password = password
        self.app_key = app_key
        self.certs_path = certs_path
        self.trading = betfairlightweight.APIClient(
            username=username,
            password=password,
            app_key=app_key,
            certs=certs_path,
        )
        self.session = requests.Session()

    def login(self):
        """Login to Betfair"""
        self.trading.login(session=self.session)
        I("Logged in to Betfair successfully")

    def logout(self):
        """Logout from Betfair"""
        self.trading.logout()
        I("Logged out from Betfair")

    def get_premier_league_competition_id(self):
        """Find the Premier League competition ID"""
        event_type_filter = betfairlightweight.filters.market_filter(
            event_type_ids=["1"],  # Soccer
            text_query="Premier League"
        )
        
        competitions = self.trading.betting.list_competitions(
            filter=event_type_filter
        )
        
        for comp in competitions:
            if "Premier League" in comp.competition.name or "English Premier League" in comp.competition.name:
                I(f"Found Premier League: {comp.competition.name} - ID: {comp.competition.id}")
                return comp.competition.id
        
        return None

    def get_football_events(self, competition_id: str, from_date: datetime, to_date: datetime):
        """Get football events for a specific competition and date range"""
        event_filter = betfairlightweight.filters.market_filter(
            event_type_ids=["1"],
            competition_ids=[competition_id],
            market_start_time={
                "from": from_date.strftime("%Y-%m-%dT%TZ"),
                "to": to_date.strftime("%Y-%m-%dT%TZ"),
            },
        )
        
        events = self.trading.betting.list_events(filter=event_filter)
        I(f"Found {len(events)} events")
        return events

    def get_market_data(self, event_id: str):
        """Get market data for a specific event"""
        market_filter = betfairlightweight.filters.market_filter(
            event_ids=[event_id],
        )
        
        market_catalogues = self.trading.betting.list_market_catalogue(
            filter=market_filter,
            max_results=100,
            market_projection=[
                'MARKET_DESCRIPTION',
                'RUNNER_METADATA',
                'EVENT',
                'MARKET_START_TIME',
                'COMPETITION'
            ],
        )
        
        return market_catalogues

    def get_market_books(self, market_ids: list[str]):
        """Get market books (prices) for specific markets"""
        price_projection = betfairlightweight.filters.price_projection(
            price_data=['EX_BEST_OFFERS', 'EX_TRADED']
        )
        
        market_books = self.trading.betting.list_market_book(
            market_ids=market_ids,
            price_projection=price_projection,
        )
        
        return market_books

    def extract_football_data(self, from_date: datetime, to_date: datetime):
        """
        Extract Premier League football data from Betfair
        """
        self.login()
        
        try:
            # Get Premier League competition ID
            competition_id = self.get_premier_league_competition_id()
            
            if not competition_id:
                D("Could not find Premier League competition")
                return pd.DataFrame()
            
            # Get events (matches)
            events = self.get_football_events(competition_id, from_date, to_date)
            
            if not events:
                I("No events found for the specified date range")
                return pd.DataFrame()
            
            # Extract market data for each event
            all_markets = []
            
            for event in events:
                event_id = event.event.id
                event_name = event.event.name
                event_date = event.event.open_date
                
                I(f"Processing event: {event_name}")
                
                # Get market catalogues
                market_catalogues = self.get_market_data(event_id)
                
                for market in market_catalogues:
                    market_id = market.market_id
                    market_name = market.market_name
                    market_type = market.description.market_type
                    
                    # Extract runner information
                    for runner in market.runners:
                        market_record = {
                            'event_id': event_id,
                            'event_name': event_name,
                            'event_date': make_uk_time_aware(event_date) if event_date else None,
                            'competition': market.competition.name if hasattr(market, 'competition') else 'Premier League',
                            'market_id': market_id,
                            'market_name': market_name,
                            'market_type': market_type,
                            'selection_id': runner.selection_id,
                            'runner_name': runner.runner_name,
                            'sort_priority': runner.sort_priority,
                        }
                        
                        all_markets.append(market_record)
                
                # Add small delay to avoid rate limiting
                sleep(0.1)
            
            df = pd.DataFrame(all_markets)
            I(f"Extracted {len(df)} market records")
            
            return df
            
        except Exception as e:
            D(f"Error extracting football data: {e}")
            raise
        
        finally:
            self.logout()

    def get_historical_prices(self, market_ids: list[str]):
        """
        Get historical prices for specific markets
        Note: This requires historical data download from Betfair
        """
        try:
            market_books = self.get_market_books(market_ids)
            
            price_data = []
            
            for market_book in market_books:
                market_id = market_book.market_id
                
                for runner in market_book.runners:
                    if runner.ex and runner.ex.available_to_back:
                        best_back = runner.ex.available_to_back[0] if runner.ex.available_to_back else None
                        best_lay = runner.ex.available_to_lay[0] if runner.ex.available_to_lay else None
                        
                        price_record = {
                            'market_id': market_id,
                            'selection_id': runner.selection_id,
                            'status': runner.status,
                            'last_price_traded': runner.last_price_traded,
                            'total_matched': runner.total_matched,
                            'back_price': best_back.price if best_back else None,
                            'back_size': best_back.size if best_back else None,
                            'lay_price': best_lay.price if best_lay else None,
                            'lay_size': best_lay.size if best_lay else None,
                        }
                        
                        price_data.append(price_record)
            
            return pd.DataFrame(price_data)
            
        except Exception as e:
            D(f"Error getting historical prices: {e}")
            raise

In [ ]:
import os
import zipfile
from pathlib import Path
import json
import bz2

class BetfairHistoricalDataDownloader:
    """
    Download and process historical data from Betfair Historical Data service
    """
    
    def __init__(self, username: str, password: str, app_key: str, certs_path: str):
        self.username = username
        self.password = password
        self.app_key = app_key
        self.certs_path = certs_path
        self.trading = betfairlightweight.APIClient(
            username=username,
            password=password,
            app_key=app_key,
            certs=certs_path,
        )
        self.session = requests.Session()
        
    def login(self):
        """Login to Betfair"""
        self.trading.login(session=self.session)
        print("✓ Logged in to Betfair successfully")
        
    def logout(self):
        """Logout from Betfair"""
        self.trading.logout()
        print("✓ Logged out from Betfair")
    
    def list_available_files(self, sport: str = "Soccer", plan: str = "Basic Plan", 
                            from_date: str = None, to_date: str = None):
        """
        List available historical data files
        
        Args:
            sport: Sport name (e.g., 'Soccer', 'Horse Racing')
            plan: Data plan type ('Basic Plan', 'Advanced Plan', 'Pro Plan')
            from_date: Start date in format 'YYYY-MM-DD'
            to_date: End date in format 'YYYY-MM-DD'
        """
        url = "https://historicdata.betfair.com/api/listFilesInformation"
        
        params = {
            "sport": sport,
            "plan": plan,
        }
        
        if from_date:
            params["fromDate"] = from_date
        if to_date:
            params["toDate"] = to_date
            
        headers = {
            "X-Application": self.app_key,
            "X-Authentication": self.trading.session_token
        }
        
        response = self.session.get(url, params=params, headers=headers)
        
        if response.status_code == 200:
            files = response.json()
            print(f"✓ Found {len(files)} available data files")
            return files
        else:
            print(f"✗ Error: {response.status_code} - {response.text}")
            return []
    
    def download_file(self, file_path: str, output_dir: str = "./historical_data"):
        """
        Download a specific historical data file
        
        Args:
            file_path: Path to the file on Betfair's servers (from list_available_files)
            output_dir: Local directory to save the file
        """
        url = "https://historicdata.betfair.com/api/downloadFile"
        
        params = {"filePath": file_path}
        
        headers = {
            "X-Application": self.app_key,
            "X-Authentication": self.trading.session_token
        }
        
        # Create output directory if it doesn't exist
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
        # Extract filename from path
        filename = file_path.split('/')[-1]
        output_path = os.path.join(output_dir, filename)
        
        print(f"Downloading {filename}...")
        
        response = self.session.get(url, params=params, headers=headers, stream=True)
        
        if response.status_code == 200:
            with open(output_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
            print(f"✓ Downloaded {filename} ({file_size:.2f} MB)")
            return output_path
        else:
            print(f"✗ Error downloading {filename}: {response.status_code}")
            return None
    
    def extract_zip(self, zip_path: str, extract_to: str = None):
        """Extract a zip file"""
        if extract_to is None:
            extract_to = os.path.dirname(zip_path)
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        
        print(f"✓ Extracted {os.path.basename(zip_path)}")
        return extract_to
    
    def read_market_file(self, file_path: str, max_records: int = None):
        """
        Read and parse a Betfair market data file (.bz2 or .json)
        
        Returns a list of market records
        """
        records = []
        
        # Handle .bz2 compressed files
        if file_path.endswith('.bz2'):
            with bz2.open(file_path, 'rt', encoding='utf-8') as f:
                for i, line in enumerate(f):
                    if max_records and i >= max_records:
                        break
                    try:
                        record = json.loads(line)
                        records.append(record)
                    except json.JSONDecodeError:
                        continue
        else:
            # Handle regular .json files
            with open(file_path, 'r', encoding='utf-8') as f:
                for i, line in enumerate(f):
                    if max_records and i >= max_records:
                        break
                    try:
                        record = json.loads(line)
                        records.append(record)
                    except json.JSONDecodeError:
                        continue
        
        print(f"✓ Read {len(records)} records from {os.path.basename(file_path)}")
        return records
    
    def parse_market_data(self, records: list):
        """
        Parse market data records into a structured DataFrame
        """
        parsed_data = []
        
        for record in records:
            # Extract market catalogue (metadata)
            if 'mc' in record:
                mc = record['mc']
                market_id = record.get('id', '')
                
                for runner in mc:
                    parsed_data.append({
                        'market_id': market_id,
                        'selection_id': runner.get('id', ''),
                        'runner_name': runner.get('name', ''),
                        'status': runner.get('status', ''),
                    })
            
            # Extract market book (price data)
            elif 'marketDefinition' in record:
                market_def = record['marketDefinition']
                market_id = record.get('id', '')
                
                for runner in market_def.get('runners', []):
                    parsed_data.append({
                        'market_id': market_id,
                        'selection_id': runner.get('id', ''),
                        'runner_name': runner.get('name', ''),
                        'status': runner.get('status', ''),
                    })
        
        return pd.DataFrame(parsed_data)
    
    def download_premier_league_data(self, from_date: str, to_date: str, 
                                     output_dir: str = "./premier_league_data",
                                     market_types: list = None):
        """
        Download Premier League historical data for a date range
        
        Args:
            from_date: Start date 'YYYY-MM-DD'
            to_date: End date 'YYYY-MM-DD'
            output_dir: Directory to save files
            market_types: List of market types to filter (optional)
        """
        if market_types is None:
            market_types = ['MATCH_ODDS', 'OVER_UNDER_25', 'CORRECT_SCORE', 
                          'BOTH_TEAMS_TO_SCORE', 'ASIAN_HANDICAP']
        
        self.login()
        
        try:
            # List available files
            print(f"\nSearching for Premier League data from {from_date} to {to_date}...")
            files = self.list_available_files(
                sport="Soccer",
                plan="Basic Plan",
                from_date=from_date,
                to_date=to_date
            )
            
            if not files:
                print("No files found for the specified date range")
                return []
            
            # Filter for Premier League files
            pl_files = [f for f in files if 'Premier League' in f.get('eventName', '') 
                       or 'English Premier League' in f.get('eventName', '')]
            
            print(f"✓ Found {len(pl_files)} Premier League files")
            
            # Download each file
            downloaded_files = []
            for file_info in pl_files:
                file_path = file_info.get('filePath', '')
                if file_path:
                    local_path = self.download_file(file_path, output_dir)
                    if local_path:
                        downloaded_files.append(local_path)
                
                # Add delay to avoid rate limiting
                sleep(0.5)
            
            print(f"\n✓ Downloaded {len(downloaded_files)} files to {output_dir}")
            return downloaded_files
            
        except Exception as e:
            print(f"✗ Error: {e}")
            raise
        
        finally:
            self.logout()

In [ ]:
# Initialize the Betfair Historical Data Downloader
downloader = BetfairHistoricalDataDownloader(
    username=config.bf_username,
    password=config.bf_password,
    app_key=config.bf_app_key,
    certs=config.bf_certs_path,
)

print("Historical Data Downloader initialized")

In [ ]:
# Login to Betfair
downloader.login()

2025-11-15T05:33:16Z | INFO - Logged in to Betfair successfully


In [ ]:
# List available files for Premier League
# This will show what historical data is available
available_files = downloader.list_available_files(
    sport="Soccer",
    plan="Basic Plan",
    from_date="2024-08-01",  # Start of 2024/25 season
    to_date="2025-05-31"     # End of season
)

print(f"\nTotal files available: {len(available_files)}")

2025-11-15T05:33:19Z | INFO - Found Premier League: Panamanian Premier League - ID: 3972877


Premier League Competition ID: 3972877


In [ ]:
# Filter for Premier League files only
if available_files:
    pl_files = [f for f in available_files 
                if 'Premier League' in str(f.get('eventName', '')) 
                or 'English Premier League' in str(f.get('eventName', ''))]
    
    print(f"Premier League files: {len(pl_files)}")
    
    # Show sample of files
    if pl_files:
        print("\nSample files:")
        for i, file in enumerate(pl_files[:5]):
            print(f"{i+1}. {file.get('eventName', 'Unknown')} - {file.get('marketType', 'Unknown')}")
            print(f"   Path: {file.get('filePath', 'N/A')}")
            print(f"   Date: {file.get('marketStartTime', 'N/A')}\n")
else:
    print("No files available - you may need a Historical Data subscription")

Fetching Premier League data from 2025-08-17 to 2025-11-15


In [ ]:
# Download Premier League historical data
# Specify date range for data you want
download_from = "2024-08-01"  # Start of season
download_to = "2024-12-31"    # Up to end of year

# Specify which market types to include
market_types = [
    'MATCH_ODDS',           # Home/Draw/Away (includes ante-post)
    'OVER_UNDER_25',        # Over/Under 2.5 goals
    'CORRECT_SCORE',        # Correct score markets
    'BOTH_TEAMS_TO_SCORE',  # BTTS
    'ASIAN_HANDICAP',       # Asian handicap
]

print(f"Starting download for {download_from} to {download_to}")
print(f"Market types: {', '.join(market_types)}\n")

downloaded_files = downloader.download_premier_league_data(
    from_date=download_from,
    to_date=download_to,
    output_dir="./premier_league_historical",
    market_types=market_types
)

2025-11-15T05:33:36Z | INFO - Found 0 events


Found 0 Premier League matches


In [ ]:
# Process downloaded files
# Read and parse the market data from downloaded files

all_market_data = []

if downloaded_files:
    for file_path in downloaded_files[:5]:  # Process first 5 files as example
        print(f"\nProcessing {os.path.basename(file_path)}...")
        
        # Extract if zip
        if file_path.endswith('.zip'):
            extract_dir = downloader.extract_zip(file_path)
            
            # Find all .bz2 or .json files in extracted directory
            data_files = list(Path(extract_dir).glob('**/*.bz2')) + \
                        list(Path(extract_dir).glob('**/*.json'))
        else:
            data_files = [file_path]
        
        # Read each data file
        for data_file in data_files:
            print(f"  Reading {data_file.name}...")
            try:
                records = downloader.read_market_file(str(data_file), max_records=100)
                
                if records:
                    # Parse the records
                    df = downloader.parse_market_data(records)
                    
                    if not df.empty:
                        df['source_file'] = data_file.name
                        all_market_data.append(df)
                        print(f"    ✓ Parsed {len(df)} records")
            except Exception as e:
                print(f"    ✗ Error reading {data_file.name}: {e}")

# Combine all data
if all_market_data:
    combined_df = pd.concat(all_market_data, ignore_index=True)
    print(f"\n✓ Total records combined: {len(combined_df)}")
    print(f"✓ Unique markets: {combined_df['market_id'].nunique()}")
else:
    print("\nNo data to process")
    combined_df = pd.DataFrame()


Fetching all markets for event: 'Fontwell 22nd Sep' (ID: 34754953)

Fetching all markets for event: 'Wolverhampton 22nd Sep' (ID: 34754958)

Fetching all markets for event: 'Hamilton 22nd Sep' (ID: 34754956)

Fetching all markets for event: 'Leicester 22nd Sep' (ID: 34754957)


In [ ]:
# Display sample of the data
if not combined_df.empty:
    print("Sample of downloaded data:")
    combined_df.head(20)
else:
    print("No data available to display")

In [ ]:
# Group by market to see the structure
if not combined_df.empty:
    market_summary = combined_df.groupby('market_id').agg({
        'selection_id': 'count',
        'runner_name': lambda x: ', '.join(x.unique()[:5]),  # First 5 runner names
        'source_file': 'first'
    }).rename(columns={
        'selection_id': 'num_runners',
        'runner_name': 'runners'
    }).head(10)
    
    print("Market summary:")
    market_summary
else:
    print("No data to summarize")

,event_name,event_id,race_id,course,market_name,market_id,race_time,market_type,race_type,selection_id,horse_name,number_of_runners,handicap,market_win_place
8,Fontwell 22nd Sep,34754953,a37de2a3182e95d41a79a0351f8c0c4a9ea3625cd89a22...,Fontwell,2m2f Nov Hcap Hrd,1.248001865,2025-09-22 14:30:00+01:00,WIN,Hurdle,41248189,Hill Station,8,True,1
9,Fontwell 22nd Sep,34754953,a37de2a3182e95d41a79a0351f8c0c4a9ea3625cd89a22...,Fontwell,2m2f Nov Hcap Hrd,1.248001865,2025-09-22 14:30:00+01:00,WIN,Hurdle,77034532,Jorebel,8,True,1
10,Fontwell 22nd Sep,34754953,a37de2a3182e95d41a79a0351f8c0c4a9ea3625cd89a22...,Fontwell,2m2f Nov Hcap Hrd,1.248001865,2025-09-22 14:30:00+01:00,WIN,Hurdle,1262915,Graham,8,True,1
11,Fontwell 22nd Sep,34754953,a37de2a3182e95d41a79a0351f8c0c4a9ea3625cd89a22...,Fontwell,2m2f Nov Hcap Hrd,1.248001865,2025-09-22 14:30:00+01:00,WIN,Hurdle,63973716,Lusso Milan,8,True,1
12,Fontwell 22nd Sep,34754953,a37de2a3182e95d41a79a0351f8c0c4a9ea3625cd89a22...,Fontwell,2m2f Nov Hcap Hrd,1.248001865,2025-09-22 14:30:00+01:00,WIN,Hurdle,45951186,Chester Tonik,8,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,Leicester 22nd Sep,34754957,a3ad600cccd25b2c233cdff59424b1599863176256f7c5...,Leicester,6f Hcap,1.248001997,2025-09-22 17:38:00+01:00,WIN,Flat,38036,Bay Breeze,15,True,1
1273,Leicester 22nd Sep,34754957,a3ad600cccd25b2c233cdff59424b1599863176256f7c5...,Leicester,6f Hcap,1.248001997,2025-09-22 17:38:00+01:00,WIN,Flat,35625781,Rough Diamond,15,True,1
1274,Leicester 22nd Sep,34754957,a3ad600cccd25b2c233cdff59424b1599863176256f7c5...,Leicester,6f Hcap,1.248001997,2025-09-22 17:38:00+01:00,WIN,Flat,70397064,Skellig Isle,15,True,1
1275,Leicester 22nd Sep,34754957,a3ad600cccd25b2c233cdff59424b1599863176256f7c5...,Leicester,6f Hcap,1.248001997,2025-09-22 17:38:00+01:00,WIN,Flat,86793021,A Rose Adaay,15,True,1


In [ ]:
# Save to CSV
if not combined_df.empty:
    output_file = 'premier_league_historical_data.csv'
    combined_df.to_csv(output_file, index=False)
    print(f"✓ Data saved to {output_file}")
    print(f"  - Total records: {len(combined_df)}")
    print(f"  - Unique markets: {combined_df['market_id'].nunique()}")
    print(f"  - Unique selections: {combined_df['selection_id'].nunique()}")
else:
    print("No data to save")

In [ ]:
# Logout
downloader.logout()

,race_id,race_time,market_name,market_type,market_id,number_of_runners
10,bbad49bbffd9106a53b26742cc1958301c82b676d7e153...,2025-09-20 13:00:00+01:00,1m Nov Stks,WIN,1.247938605,8
1645,76c42e40ae100950a6e910c11f339a538a4333afa646b8...,2025-09-20 13:15:00+01:00,1m Hcap,WIN,1.247939515,14
866,0fbd6237fcab334a7eb5f66c7af4682cb6a7247c552883...,2025-09-20 13:30:00+01:00,5f Grp 3,WIN,1.247938537,11
53,d198efcea63bcce385801bc808a70d306b752e75520db8...,2025-09-20 13:35:00+01:00,1m Hcap,WIN,1.247938612,9
1705,a6497d75dc1b99fd8aceebd7318ed3b2c9587911a0b9e5...,2025-09-20 13:50:00+01:00,1m2f Listed,WIN,1.247939521,6
2632,0f65e1be005404ee25513dd8fd1c20c45c1bffc86c195a...,2025-09-20 13:55:00+01:00,7f Nursery,WIN,1.247938489,9
928,e20a8f210633e57855320c79848bc687f6946442ce845b...,2025-09-20 14:05:00+01:00,1m5f Hcap,WIN,1.247938543,14
89,a3ee1a158be7af287644c0d2360c5149b7713d5bebdf28...,2025-09-20 14:10:00+01:00,4 TBP,OTHER_PLACE,1.247938713,13
1729,e0fa746d0468889f07fa16729772d6f4deaf64c1bc3908...,2025-09-20 14:25:00+01:00,5 TBP,OTHER_PLACE,1.247939532,25
2670,e77a631d029e51db1144b88a6f79ceb95c09eb0b6ee37a...,2025-09-20 14:30:00+01:00,6f Nov Stks,WIN,1.247938495,6


## Ante-Post Markets

Ante-post markets are markets that open well before the event starts (e.g., betting on match outcomes days or weeks in advance). The historical data includes price movements from when the market opens until it closes.

In [ ]:
# Download ante-post markets specifically
# These are markets that opened well before the match

downloader.login()

# For ante-post, you typically want data from several days/weeks before matches
antepost_from = "2024-08-01"
antepost_to = "2024-12-31"

print("Downloading ante-post market data...")
print(f"Date range: {antepost_from} to {antepost_to}")
print("\nNote: Ante-post markets are identified by their opening time being")
print("significantly before the event start time (hours/days in advance)\n")

antepost_files = downloader.list_available_files(
    sport="Soccer",
    plan="Basic Plan",
    from_date=antepost_from,
    to_date=antepost_to
)

# Filter for Premier League
if antepost_files:
    pl_antepost = [f for f in antepost_files 
                   if 'Premier League' in str(f.get('eventName', '')) 
                   or 'English Premier League' in str(f.get('eventName', ''))]
    
    print(f"✓ Found {len(pl_antepost)} Premier League files (includes ante-post)")
    
    # You can further filter by checking if marketStartTime is before eventStartTime
    # or by looking at specific market types that are typically ante-post
else:
    print("No files available")

downloader.logout()

## Advanced: Parsing Price Data

Historical data files contain detailed price movements over time. Each record includes timestamps and price ladders.

In [ ]:
def parse_detailed_price_data(records):
    """
    Parse detailed price data including timestamps and price movements
    
    Returns a DataFrame with price information at each timestamp
    """
    price_data = []
    
    for record in records:
        timestamp = record.get('pt', 0)  # Publish time
        market_id = record.get('id', '')
        
        # Check if this is a market book update (price data)
        if 'mc' in record:
            # Market catalogue (metadata)
            continue
        
        # Parse runner price data
        runners = record.get('rc', [])
        
        for runner in runners:
            selection_id = runner.get('id', '')
            
            # Get best back and lay prices
            back_prices = runner.get('batb', [])  # Best Available To Back
            lay_prices = runner.get('batl', [])   # Best Available To Lay
            
            # Get traded volume
            traded_volume = runner.get('tv', 0)
            
            # Last price traded
            lpt = runner.get('lpt', None)
            
            price_record = {
                'timestamp': timestamp,
                'market_id': market_id,
                'selection_id': selection_id,
                'last_price_traded': lpt,
                'traded_volume': traded_volume,
            }
            
            # Add best back prices (up to 3 levels)
            for i, back in enumerate(back_prices[:3]):
                price_record[f'back_price_{i+1}'] = back[0] if len(back) > 0 else None
                price_record[f'back_size_{i+1}'] = back[1] if len(back) > 1 else None
            
            # Add best lay prices (up to 3 levels)
            for i, lay in enumerate(lay_prices[:3]):
                price_record[f'lay_price_{i+1}'] = lay[0] if len(lay) > 0 else None
                price_record[f'lay_size_{i+1}'] = lay[1] if len(lay) > 1 else None
            
            price_data.append(price_record)
    
    df = pd.DataFrame(price_data)
    
    # Convert timestamp to datetime
    if 'timestamp' in df.columns and not df.empty:
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    return df

# Example usage:
# if downloaded_files:
#     sample_file = downloaded_files[0]
#     records = downloader.read_market_file(sample_file, max_records=1000)
#     price_df = parse_detailed_price_data(records)
#     print(f"Parsed {len(price_df)} price records")
#     price_df.head()

print("Price parsing function defined. Use it to extract detailed price movements.")

In [ ]:
# Helper function to identify ante-post markets
def identify_antepost_markets(market_data_df, hours_threshold=2):
    """
    Identify ante-post markets based on time between market open and event start
    
    Args:
        market_data_df: DataFrame with market metadata
        hours_threshold: Minimum hours before event to be considered ante-post
    
    Returns:
        DataFrame with is_antepost flag
    """
    if 'market_start_time' in market_data_df.columns and 'event_start_time' in market_data_df.columns:
        market_data_df['market_start_time'] = pd.to_datetime(market_data_df['market_start_time'])
        market_data_df['event_start_time'] = pd.to_datetime(market_data_df['event_start_time'])
        
        # Calculate hours between market open and event start
        time_diff = (market_data_df['event_start_time'] - market_data_df['market_start_time']).dt.total_seconds() / 3600
        
        # Mark as ante-post if market opened X hours or more before event
        market_data_df['is_antepost'] = time_diff >= hours_threshold
        market_data_df['hours_before_event'] = time_diff
        
        print(f"Ante-post markets: {market_data_df['is_antepost'].sum()}")
        print(f"In-play/near-time markets: {(~market_data_df['is_antepost']).sum()}")
    else:
        print("Cannot identify ante-post - missing time columns")
        market_data_df['is_antepost'] = False
    
    return market_data_df

print("Ante-post identification function defined.")

## Complete Workflow Example

Here's the complete workflow to download and process Premier League historical data including ante-post markets:

1. **Initialize** - Create downloader instance with credentials
2. **Login** - Authenticate with Betfair
3. **List Files** - Check available historical data files
4. **Download** - Download files for your date range
5. **Parse** - Extract and parse market data
6. **Identify Ante-Post** - Flag markets that opened early
7. **Analyze** - Process price movements and statistics
8. **Save** - Export to CSV/database for further analysis

In [ ]:
# Save to CSV
output_file = 'premier_league_betfair_data.csv'
df_markets.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")

# Also save match odds separately
match_odds_file = 'premier_league_match_odds.csv'
match_odds.to_csv(match_odds_file, index=False)
print(f"Match odds data saved to {match_odds_file}")

df_markets.head()

,race_id,market_name,market_type,number_of_runners
10,bbad49bbffd9106a53b26742cc1958301c82b676d7e153...,1m Nov Stks,WIN,8
26,bbad49bbffd9106a53b26742cc1958301c82b676d7e153...,2 TBP,OTHER_PLACE,8
34,bbad49bbffd9106a53b26742cc1958301c82b676d7e153...,4 TBP,OTHER_PLACE,8
18,bbad49bbffd9106a53b26742cc1958301c82b676d7e153...,To Be Placed,PLACE,8
1645,76c42e40ae100950a6e910c11f339a538a4333afa646b8...,1m Hcap,WIN,14
1673,76c42e40ae100950a6e910c11f339a538a4333afa646b8...,2 TBP,OTHER_PLACE,14
1687,76c42e40ae100950a6e910c11f339a538a4333afa646b8...,4 TBP,OTHER_PLACE,14
1659,76c42e40ae100950a6e910c11f339a538a4333afa646b8...,To Be Placed,PLACE,14
888,0fbd6237fcab334a7eb5f66c7af4682cb6a7247c552883...,2 TBP,OTHER_PLACE,11
899,0fbd6237fcab334a7eb5f66c7af4682cb6a7247c552883...,4 TBP,OTHER_PLACE,11


## Data Analysis

In [ ]:
# Analyze market types distribution
market_type_counts = df_markets.groupby('market_type').agg({
    'event_id': 'nunique',
    'market_id': 'nunique',
    'selection_id': 'count'
}).rename(columns={
    'event_id': 'num_matches',
    'market_id': 'num_markets',
    'selection_id': 'num_selections'
})

print("Market Types Summary:")
market_type_counts

In [ ]:
# Visualize matches over time
import matplotlib.pyplot as plt

df_markets_with_date = df_markets.copy()
df_markets_with_date['date_only'] = pd.to_datetime(df_markets_with_date['event_date']).dt.date

matches_per_day = df_markets_with_date.groupby('date_only')['event_id'].nunique()

plt.figure(figsize=(14, 6))
plt.bar(range(len(matches_per_day)), matches_per_day.values)
plt.xlabel('Date')
plt.ylabel('Number of Matches')
plt.title('Premier League Matches Over Time')
plt.xticks(range(0, len(matches_per_day), max(1, len(matches_per_day)//10)), 
           [str(d) for d in matches_per_day.index[::max(1, len(matches_per_day)//10)]], 
           rotation=45)
plt.tight_layout()
plt.show()

## Alternative: Using Historical Data Download

For complete historical data (past seasons), Betfair provides a historical data service. You'll need to:
1. Subscribe to Betfair Historical Data
2. Download data files from their portal
3. Process the files locally

The code below shows how to set up parameters for historical data download.

In [ ]:
# Example: Historical data download parameters for Premier League
# Note: This requires subscription to Betfair Historical Data service

historical_params = BetfairHistoricalDataParams(
    from_day=1,
    from_month=8,  # August (start of season)
    from_year=2024,
    to_day=31,
    to_month=5,  # May (end of season)
    to_year=2025,
    market_types_collection=['MATCH_ODDS', 'OVER_UNDER_25', 'BOTH_TEAMS_TO_SCORE'],
    countries_collection=['GB'],
    file_type_collection=['M']  # Market data
)

print("Historical Data Parameters:")
print(f"Date Range: {historical_params.from_day}/{historical_params.from_month}/{historical_params.from_year} to {historical_params.to_day}/{historical_params.to_month}/{historical_params.to_year}")
print(f"Market Types: {', '.join(historical_params.market_types_collection)}")
print(f"Countries: {', '.join(historical_params.countries_collection)}")

In [ ]:
# Logout from Betfair
football_client.logout()
print("Logged out successfully")

# Premier League Historical Data Analysis

This notebook pulls historical Premier League football data from Betfair, including:
- Match information (teams, dates)
- Market types (Match Odds, Over/Under, etc.)
- Selection IDs and runner names
- Optional: Current prices for active markets

In [59]:
df['market_name'].value_counts()

market_name
To Be Placed     431
4 TBP            355
2 TBP            325
6f Hcap           83
3 TBP             76
7f Hcap           74
6 TBP             50
5 TBP             50
1m Hcap           38
1m1f Hcap         26
7f Mdn Stks       24
7f Nov Stks       20
1m2f Hcap         20
5f Hcap           19
6f Grp 3          15
1m5f Hcap         14
1m6f Hcap         14
5f Grp 3          11
1m2f Nov Stks      9
7f Nursery         9
7f Stks            9
2m2f Hcap          9
6f Grp 2           8
5f Nursery         8
1m Nov Stks        8
1m4f Hcap          7
1m2f Listed        6
6f Nov Stks        6
1m1f Nursery       3
Name: count, dtype: int64

In [42]:
def calculate_win_place_market_type(market_name: str, number_of_runners: int) -> int:
    """
    Calculate the number of places for a 'To Be Placed' market based on the market name and number of runners.

    Args:
        market_name (str): The name of the market, e.g., "To Be Placed 1-3 (4+ Runners) TBP".
        number_of_runners (int): The total number

    Returns:
        int: The number of places for the market.

    """

    

market_name
To Be Placed     38
4 TBP            32
2 TBP            30
3 TBP             6
7f Hcap           6
6f Hcap           5
1m Hcap           3
7f Mdn Stks       2
1m2f Hcap         2
5 TBP             2
6 TBP             2
1m1f Hcap         2
7f Nov Stks       2
5f Hcap           2
7f Nursery        1
6f Grp 3          1
6f Nov Stks       1
7f Stks           1
1m2f Listed       1
1m Nov Stks       1
6f Grp 2          1
1m5f Hcap         1
5f Grp 3          1
1m4f Hcap         1
5f Nursery        1
1m1f Nursery      1
1m2f Nov Stks     1
2m2f Hcap         1
1m6f Hcap         1
Name: count, dtype: int64

In [43]:
df.to_csv('~/Desktop/test.csv')

In [118]:
markets = []
if events:
    # Loop through each event we found in the initial search
    for event in events:
        event_id = event.event.id
        event_name = event.event.name
        print(f"\nFetching all markets for event: '{event_name}' (ID: {event_id})")

        # Create a filter for the markets of the current event in the loop
        market_catalogue_filter = filters.market_filter(
            event_ids=[event_id],
        )

        # Request the market catalogue for this specific event
        market_catalogues = trading.betting.list_market_catalogue(
            filter=market_catalogue_filter,
            max_results=100,  # Max number of markets to return per event
            market_projection=['MARKET_DESCRIPTION', 'RUNNER_METADATA']
        )

        for market in market_catalogues:
            for runner in market.runners:

                markets.append(
                    {
                        "event_name": event_name,
                        "event_id": event_id,
                        "market_name": market.market_name,
                        "market_id": market.market_id,
                        'race_time': market.market_start_time,
                        "market_type": market.description.market_type,
                        "selection_id": runner.selection_id,
                        "horse_name": runner.runner_name,
                    }
            )   



Fetching all markets for event: 'Southwell 21st Sep' (ID: 34751698)

Fetching all markets for event: 'Hamilton 21st Sep' (ID: 34751687)

Fetching all markets for event: 'Plumpton 21st Sep' (ID: 34751691)

Fetching all markets for event: 'Plumpton 21st Sep' (ID: 34751691)


In [ ]:
# Create a SHA-256 unique_id from race_time and event_id
import hashlib
import pandas as pd

def add_unique_id_from_race_time_and_event_id(
    df: pd.DataFrame,
    race_time_col: str = "race_time",
    event_id_col: str = "event_id",
    tz: str | None = None,
 ) -> pd.DataFrame:
    """
    Return a copy of df with a new 'unique_id' column computed as:
        sha256(f"{race_time:%Y%m%d%H%M%S}|{event_id}")
    
    Args:
        df: Input DataFrame containing race_time and event_id.
        race_time_col: Name of the race_time column.
        event_id_col: Name of the event_id column.
        tz: Optional timezone name (e.g., 'Europe/London') to normalize race_time before hashing.
    """
    d = df.copy()
    rt = pd.to_datetime(d[race_time_col], errors="coerce")
    
    # Optionally normalize timezone for deterministic hashing
    if tz is not None:
        try:
            if rt.dt.tz is None:
                rt = rt.dt.tz_localize(tz)
            else:
                rt = rt.dt.tz_convert(tz)
        except Exception:
            # If tz conversion/localization fails, fall back to original parsed times
            pass
    
    race_time_str = rt.dt.strftime("%Y%m%d%H%M%S").fillna("")
    key_series = race_time_str + "|" + d[event_id_col].astype(str).fillna("")
    d["unique_id"] = key_series.map(lambda s: hashlib.sha256(s.encode("utf-8")).hexdigest())
    return d

# Example usage:
# df = add_unique_id_from_race_time_and_event_id(df)


In [ ]:
{
    "marketId": "1.247969675",
    "marketName": "To Be Placed",
    "marketStartTime": "2025-09-21T14:07:00.000Z",
    "description": {
        "persistenceEnabled": True,
        "bspMarket": True,
        "marketTime": "2025-09-21T14:07:00.000Z",
        "suspendTime": "2025-09-21T14:07:00.000Z",
        "bettingType": "ODDS",
        "turnInPlayEnabled": True,
        "marketType": "PLACE",
        "regulator": "MALTA LOTTERIES AND GAMBLING AUTHORITY",
        "marketBaseRate": 2.0,
        "discountAllowed": False,
        "wallet": "UK wallet",
        "rules": '<br><a href="https://www.timeform.com/horse-racing/" target="_blank"><img src=" http://content-cache.betfair.com/images/en_GB/mr_fr.gif" title=”Form/ Results” border="0"></a>\n<br><br><b>MARKET INFORMATION</b><br><br>For further information please see <a href=http://content.betfair.com/aboutus/content.asp?sWhichKey=Rules%20and%20Regulations#undefined.do style=color:0163ad; text-decoration: underline; target=_blank>Rules & Regs.</a><br><br>Who will finish 1st, 2nd or 3rd in this race? NON RUNNERS DO NOT CHANGE THE PLACE TERMS. Should the number of runners be equal to or less than the number of places available as set out above in these rules all bets will be void. Betfair Non-Runner Rule applies. <b>This market will turn IN PLAY at the off with unmatched bets (with the exception of bets for which the "keep" option has been selected) cancelled once the Betfair SP reconciliation process has been completed. Betting will be suspended at the end of the race.</b> This market will initially be settled on a First Past the Post basis. However we will re-settle all bets should the official result at the time of the "weigh-in" announcement differ from any initial settlement. BETS ARE PLACED ON A NAMED HORSE. Dead Heat rules apply.<br><br>Customers should be aware that:<ol><b><li>transmissions described as "live" by some broadcasters may actually be delayed;</li><li>the extent of any such delay may vary, depending on the set-up through which they are receiving pictures or data; and</li></b><li>information (such as jockey silks, saddlecloth numbers etc) is provided "as is" and is for guidance only. Betfair does not guarantee the accuracy of this information and use of it to place bets is entirely at your own risk.</li></ol><br>',
        "rulesHasDate": True,
        "raceType": "Flat",
        "priceLadderDescription": {"type": "CLASSIC"},
    },
    "totalMatched": 2.1,
    "runners": [
        {
            "selectionId": 50612968,
            "runnerName": "Dragon Icon",
            "handicap": 0.0,
            "sortPriority": 1,
            "metadata": {
                "SIRE_NAME": "Lope De Vega",
                "CLOTH_NUMBER_ALPHA": "2",
                "OFFICIAL_RATING": "81",
                "COLOURS_DESCRIPTION": "Blue, red sleeves",
                "COLOURS_FILENAME": "c20250921sou/00881392.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Hurricane Run",
                "WEIGHT_VALUE": "136",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "15",
                "WEARING": "Cheek pieces",
                "OWNER_NAME": "Mrs Angelica Spence",
                "DAM_YEAR_BORN": "2009",
                "SIRE_BRED": "IRL",
                "JOCKEY_NAME": "Ray Dawson",
                "DAM_BRED": "IRL",
                "ADJUSTED_RATING": None,
                "runnerId": "50612968",
                "CLOTH_NUMBER": "2",
                "SIRE_YEAR_BORN": "2007",
                "TRAINER_NAME": "R Varian",
                "COLOUR_TYPE": "b",
                "AGE": "5",
                "DAMSIRE_BRED": "IRL",
                "JOCKEY_CLAIM": "0",
                "FORM": "350733",
                "FORECASTPRICE_NUMERATOR": "6",
                "BRED": None,
                "DAM_NAME": "Matauri Pearl",
                "DAMSIRE_YEAR_BORN": "2002",
                "STALL_DRAW": "1",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 74754399,
            "runnerName": "Best Rate",
            "handicap": 0.0,
            "sortPriority": 2,
            "metadata": {
                "SIRE_NAME": "Camacho",
                "CLOTH_NUMBER_ALPHA": "10",
                "OFFICIAL_RATING": "82",
                "COLOURS_DESCRIPTION": "Black and yellow diamonds, black sleeves, yellow cap",
                "COLOURS_FILENAME": "c20250921sou/00881778.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Royal Applause",
                "WEIGHT_VALUE": "133",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "16",
                "WEARING": None,
                "OWNER_NAME": "Jonathan Palmer-brown And Jim Horgan",
                "DAM_YEAR_BORN": "2010",
                "SIRE_BRED": "GBR",
                "JOCKEY_NAME": "Tom Marquand",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "74754399",
                "CLOTH_NUMBER": "10",
                "SIRE_YEAR_BORN": "2002",
                "TRAINER_NAME": "R Hannon",
                "COLOUR_TYPE": "b",
                "AGE": "3",
                "DAMSIRE_BRED": "GBR",
                "JOCKEY_CLAIM": "0",
                "FORM": "204423",
                "FORECASTPRICE_NUMERATOR": "6",
                "BRED": None,
                "DAM_NAME": "Nardin",
                "DAMSIRE_YEAR_BORN": "1993",
                "STALL_DRAW": "7",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 47296230,
            "runnerName": "Rajapour",
            "handicap": 0.0,
            "sortPriority": 3,
            "metadata": {
                "SIRE_NAME": "Almanzor",
                "CLOTH_NUMBER_ALPHA": "8",
                "OFFICIAL_RATING": "79",
                "COLOURS_DESCRIPTION": "White, red inverted triangle, hooped sleeves and cap",
                "COLOURS_FILENAME": "c20250921sou/00062867.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Rock Of Gibraltar",
                "WEIGHT_VALUE": "134",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "14",
                "WEARING": None,
                "OWNER_NAME": "Mr Evan M Sutherland",
                "DAM_YEAR_BORN": "2012",
                "SIRE_BRED": "FRA",
                "JOCKEY_NAME": "Mark Winn",
                "DAM_BRED": "IRL",
                "ADJUSTED_RATING": None,
                "runnerId": "47296230",
                "CLOTH_NUMBER": "8",
                "SIRE_YEAR_BORN": "1994",
                "TRAINER_NAME": "D O'Meara",
                "COLOUR_TYPE": "ch",
                "AGE": "5",
                "DAMSIRE_BRED": "IRL",
                "JOCKEY_CLAIM": "0",
                "FORM": "6769-42",
                "FORECASTPRICE_NUMERATOR": "7",
                "BRED": None,
                "DAM_NAME": "Raydara",
                "DAMSIRE_YEAR_BORN": "1999",
                "STALL_DRAW": "11",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 71597397,
            "runnerName": "Degale",
            "handicap": 0.0,
            "sortPriority": 4,
            "metadata": {
                "SIRE_NAME": "Due Diligence",
                "CLOTH_NUMBER_ALPHA": "4",
                "OFFICIAL_RATING": "80",
                "COLOURS_DESCRIPTION": "White, emerald green cross of lorraine,  chevrons on sleeves, pink cap",
                "COLOURS_FILENAME": "c20250921sou/00848182.jpg",
                "FORECASTPRICE_DENOMINATOR": "2",
                "DAMSIRE_NAME": "Broken Vow",
                "WEIGHT_VALUE": "135",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "42",
                "WEARING": None,
                "OWNER_NAME": "Mrlaurenceo'kane/harrowgatebloodstockl",
                "DAM_YEAR_BORN": "1998",
                "SIRE_BRED": "USA",
                "JOCKEY_NAME": "Callum Rodriguez",
                "DAM_BRED": "USA",
                "ADJUSTED_RATING": None,
                "runnerId": "71597397",
                "CLOTH_NUMBER": "4",
                "SIRE_YEAR_BORN": "2011",
                "TRAINER_NAME": "T D Barron",
                "COLOUR_TYPE": "b",
                "AGE": "4",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "831-2",
                "FORECASTPRICE_NUMERATOR": "11",
                "BRED": None,
                "DAM_NAME": "Nuptials",
                "DAMSIRE_YEAR_BORN": "1997",
                "STALL_DRAW": "6",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 57120811,
            "runnerName": "My Margie",
            "handicap": 0.0,
            "sortPriority": 5,
            "metadata": {
                "SIRE_NAME": "Dandy Man",
                "CLOTH_NUMBER_ALPHA": "5",
                "OFFICIAL_RATING": "80",
                "COLOURS_DESCRIPTION": "Red, large yellow spots, armlets and spots on cap",
                "COLOURS_FILENAME": "c20250921sou/00885004.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Dynaformer",
                "WEIGHT_VALUE": "135",
                "SEX_TYPE": "F",
                "DAYS_SINCE_LAST_RUN": "25",
                "WEARING": None,
                "OWNER_NAME": "Mrs M Gander",
                "DAM_YEAR_BORN": "2007",
                "SIRE_BRED": "IRL",
                "JOCKEY_NAME": "Finley Marsh",
                "DAM_BRED": "USA",
                "ADJUSTED_RATING": None,
                "runnerId": "57120811",
                "CLOTH_NUMBER": "5",
                "SIRE_YEAR_BORN": "2003",
                "TRAINER_NAME": "R Hughes",
                "COLOUR_TYPE": "b",
                "AGE": "4",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "165304",
                "FORECASTPRICE_NUMERATOR": "16",
                "BRED": None,
                "DAM_NAME": "Agapantha",
                "DAMSIRE_YEAR_BORN": "1985",
                "STALL_DRAW": "3",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 394588,
            "runnerName": "Great Blasket",
            "handicap": 0.0,
            "sortPriority": 6,
            "metadata": {
                "SIRE_NAME": "Gregorian",
                "CLOTH_NUMBER_ALPHA": "9",
                "OFFICIAL_RATING": "79",
                "COLOURS_DESCRIPTION": "Orange, orange and black check sleeves",
                "COLOURS_FILENAME": "c20250921sou/00881512.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Oasis Dream",
                "WEIGHT_VALUE": "134",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "22",
                "WEARING": "Blinkers",
                "OWNER_NAME": "Urloxhey Racing",
                "DAM_YEAR_BORN": "0",
                "SIRE_BRED": "IRL",
                "JOCKEY_NAME": "Hollie Doyle",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "394588",
                "CLOTH_NUMBER": "9",
                "SIRE_YEAR_BORN": "1997",
                "TRAINER_NAME": "Dr R Newland & J Insole",
                "COLOUR_TYPE": "b",
                "AGE": "5",
                "DAMSIRE_BRED": "GBR",
                "JOCKEY_CLAIM": "0",
                "FORM": "018163",
                "FORECASTPRICE_NUMERATOR": "7",
                "BRED": None,
                "DAM_NAME": "Dream Belle",
                "DAMSIRE_YEAR_BORN": "2000",
                "STALL_DRAW": "10",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 71352621,
            "runnerName": "Man Of Desert",
            "handicap": 0.0,
            "sortPriority": 7,
            "metadata": {
                "SIRE_NAME": "Study Of Man",
                "CLOTH_NUMBER_ALPHA": "13",
                "OFFICIAL_RATING": "75",
                "COLOURS_DESCRIPTION": "White, red hollow box",
                "COLOURS_FILENAME": "c20250921sou/00067209.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Green Desert",
                "WEIGHT_VALUE": "130",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "59",
                "WEARING": None,
                "OWNER_NAME": "Strawberry Fields Stud",
                "DAM_YEAR_BORN": "2009",
                "SIRE_BRED": "IRL",
                "JOCKEY_NAME": "Dylan Hogan",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "71352621",
                "CLOTH_NUMBER": "13",
                "SIRE_YEAR_BORN": "2015",
                "TRAINER_NAME": "C A Dwyer",
                "COLOUR_TYPE": "b",
                "AGE": "4",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "6-63048",
                "FORECASTPRICE_NUMERATOR": "14",
                "BRED": None,
                "DAM_NAME": "Desert Berry",
                "DAMSIRE_YEAR_BORN": "1983",
                "STALL_DRAW": "4",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 23983741,
            "runnerName": "Dutch Decoy",
            "handicap": 0.0,
            "sortPriority": 8,
            "metadata": {
                "SIRE_NAME": "Dutch Art",
                "CLOTH_NUMBER_ALPHA": "1",
                "OFFICIAL_RATING": "81",
                "COLOURS_DESCRIPTION": "Mauve, black chevrons on sleeves, black cap",
                "COLOURS_FILENAME": "c20250921sou/00863054.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Foxhound",
                "WEIGHT_VALUE": "136",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "31",
                "WEARING": "Cheek pieces",
                "OWNER_NAME": "Owners Group 052",
                "DAM_YEAR_BORN": "2003",
                "SIRE_BRED": "GBR",
                "JOCKEY_NAME": "J Mitchell",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "23983741",
                "CLOTH_NUMBER": "1",
                "SIRE_YEAR_BORN": "2004",
                "TRAINER_NAME": "C Johnston",
                "COLOUR_TYPE": "ch",
                "AGE": "8",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "110050",
                "FORECASTPRICE_NUMERATOR": "12",
                "BRED": None,
                "DAM_NAME": "The Terrier",
                "DAMSIRE_YEAR_BORN": "1991",
                "STALL_DRAW": "2",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 48298473,
            "runnerName": "Leadenhall",
            "handicap": 0.0,
            "sortPriority": 9,
            "metadata": {
                "SIRE_NAME": "Kingman",
                "CLOTH_NUMBER_ALPHA": "12",
                "OFFICIAL_RATING": "76",
                "COLOURS_DESCRIPTION": "Pink, dark blue star, dark blue cap",
                "COLOURS_FILENAME": "c20250921sou/00867647.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Danehill",
                "WEIGHT_VALUE": "131",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "1",
                "WEARING": None,
                "OWNER_NAME": "The Wolf Pack 2 And Partner",
                "DAM_YEAR_BORN": "2004",
                "SIRE_BRED": "GBR",
                "JOCKEY_NAME": "Sean Kirrane",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "48298473",
                "CLOTH_NUMBER": "12",
                "SIRE_YEAR_BORN": "2011",
                "TRAINER_NAME": "T D Easterby",
                "COLOUR_TYPE": "b",
                "AGE": "5",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "041446",
                "FORECASTPRICE_NUMERATOR": "8",
                "BRED": None,
                "DAM_NAME": "Promising Lead",
                "DAMSIRE_YEAR_BORN": "1986",
                "STALL_DRAW": "5",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 47648763,
            "runnerName": "Chalk Mountain",
            "handicap": 0.0,
            "sortPriority": 10,
            "metadata": {
                "SIRE_NAME": "Outstrip",
                "CLOTH_NUMBER_ALPHA": "14",
                "OFFICIAL_RATING": "75",
                "COLOURS_DESCRIPTION": "Maroon, beige chevron, sleeves and star on cap",
                "COLOURS_FILENAME": "c20250921sou/00874683.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Oasis Dream",
                "WEIGHT_VALUE": "130",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "16",
                "WEARING": "Tongue strap,Visor",
                "OWNER_NAME": "The Chalk Mountain Partnership",
                "DAM_YEAR_BORN": "2010",
                "SIRE_BRED": "GBR",
                "JOCKEY_NAME": "Rob Hornby",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "47648763",
                "CLOTH_NUMBER": "14",
                "SIRE_YEAR_BORN": "2011",
                "TRAINER_NAME": "W S Kittow",
                "COLOUR_TYPE": "gr",
                "AGE": "5",
                "DAMSIRE_BRED": "GBR",
                "JOCKEY_CLAIM": "0",
                "FORM": "234774",
                "FORECASTPRICE_NUMERATOR": "12",
                "BRED": None,
                "DAM_NAME": "Perfect Muse",
                "DAMSIRE_YEAR_BORN": "2000",
                "STALL_DRAW": "13",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 42717141,
            "runnerName": "Borgi",
            "handicap": 0.0,
            "sortPriority": 11,
            "metadata": {
                "SIRE_NAME": "Anjaal",
                "CLOTH_NUMBER_ALPHA": "3",
                "OFFICIAL_RATING": "81",
                "COLOURS_DESCRIPTION": "Black, yellow seams and armlets",
                "COLOURS_FILENAME": "c20250921sou/00005751.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Olden Times",
                "WEIGHT_VALUE": "136",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "38",
                "WEARING": None,
                "OWNER_NAME": "Mr S P C Woods",
                "DAM_YEAR_BORN": "1995",
                "SIRE_BRED": "GBR",
                "JOCKEY_NAME": "Jack Callan",
                "DAM_BRED": "IRL",
                "ADJUSTED_RATING": None,
                "runnerId": "42717141",
                "CLOTH_NUMBER": "3",
                "SIRE_YEAR_BORN": "2011",
                "TRAINER_NAME": "S Woods",
                "COLOUR_TYPE": "b",
                "AGE": "6",
                "DAMSIRE_BRED": "GBR",
                "JOCKEY_CLAIM": "5",
                "FORM": "410-596",
                "FORECASTPRICE_NUMERATOR": "20",
                "BRED": None,
                "DAM_NAME": "One Time",
                "DAMSIRE_YEAR_BORN": "1998",
                "STALL_DRAW": "8",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 48500511,
            "runnerName": "Caragio",
            "handicap": 0.0,
            "sortPriority": 12,
            "metadata": {
                "SIRE_NAME": "Caravaggio",
                "CLOTH_NUMBER_ALPHA": "11",
                "OFFICIAL_RATING": "77",
                "COLOURS_DESCRIPTION": "White, red stars, diabolo on sleeves and star on cap",
                "COLOURS_FILENAME": "c20250921sou/00881965.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "More Than Ready",
                "WEIGHT_VALUE": "132",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "485",
                "WEARING": None,
                "OWNER_NAME": "The Poor Sexy Farmers Club",
                "DAM_YEAR_BORN": "2007",
                "SIRE_BRED": "USA",
                "JOCKEY_NAME": "Tyler Heard",
                "DAM_BRED": "USA",
                "ADJUSTED_RATING": None,
                "runnerId": "48500511",
                "CLOTH_NUMBER": "11",
                "SIRE_YEAR_BORN": "2014",
                "TRAINER_NAME": "Martin Dunne",
                "COLOUR_TYPE": "gr",
                "AGE": "5",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "187/400-",
                "FORECASTPRICE_NUMERATOR": "25",
                "BRED": None,
                "DAM_NAME": "Freddies Girl",
                "DAMSIRE_YEAR_BORN": "1997",
                "STALL_DRAW": "9",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 51494,
            "runnerName": "Kalamunda",
            "handicap": 0.0,
            "sortPriority": 13,
            "metadata": {
                "SIRE_NAME": "Zoustar",
                "CLOTH_NUMBER_ALPHA": "7",
                "OFFICIAL_RATING": "79",
                "COLOURS_DESCRIPTION": "White, maroon stars, maroon sleeves, white stars, white cap, maroon star",
                "COLOURS_FILENAME": "c20250921sou/00860614.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "War Chant",
                "WEIGHT_VALUE": "134",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "29",
                "WEARING": "Cheek pieces",
                "OWNER_NAME": "Trevor And Ruth Milner",
                "DAM_YEAR_BORN": "2002",
                "SIRE_BRED": "AUS",
                "JOCKEY_NAME": "Daniel Muscutt",
                "DAM_BRED": "USA",
                "ADJUSTED_RATING": None,
                "runnerId": "51494",
                "CLOTH_NUMBER": "7",
                "SIRE_YEAR_BORN": "2010",
                "TRAINER_NAME": "J Parr",
                "COLOUR_TYPE": "b",
                "AGE": "5",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "454044",
                "FORECASTPRICE_NUMERATOR": "20",
                "BRED": None,
                "DAM_NAME": "Karens Caper",
                "DAMSIRE_YEAR_BORN": "1997",
                "STALL_DRAW": "14",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 56809975,
            "runnerName": "Zryan",
            "handicap": 0.0,
            "sortPriority": 14,
            "metadata": {
                "SIRE_NAME": "Night Of Thunder",
                "CLOTH_NUMBER_ALPHA": "6",
                "OFFICIAL_RATING": "80",
                "COLOURS_DESCRIPTION": "Emerald green, white hoops, white sleeves, emerald green diamonds, white cap, emerald green diamond",
                "COLOURS_FILENAME": "c20250921sou/00836517.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Green Desert",
                "WEIGHT_VALUE": "135",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "14",
                "WEARING": None,
                "OWNER_NAME": "Gallop Racing",
                "DAM_YEAR_BORN": "2006",
                "SIRE_BRED": "IRL",
                "JOCKEY_NAME": "James Doyle",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "56809975",
                "CLOTH_NUMBER": "6",
                "SIRE_YEAR_BORN": "2011",
                "TRAINER_NAME": "D O'Meara",
                "COLOUR_TYPE": "b",
                "AGE": "4",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "149040",
                "FORECASTPRICE_NUMERATOR": "14",
                "BRED": None,
                "DAM_NAME": "Dahama",
                "DAMSIRE_YEAR_BORN": "1983",
                "STALL_DRAW": "12",
                "WEIGHT_UNITS": "pounds",
            },
        },
    ],
    "event": {
        "id": "34751698",
        "name": "Southwell 21st Sep",
        "countryCode": "GB",
        "timezone": "Europe/London",
        "venue": "Southwell",
        "openDate": "2025-09-21T13:37:00.000Z",
    },
}